In [ ]:
### Esto fue usado en Fabric por eso no está la configuración de Spark

In [ ]:
# Importaciones de bibliotecas generales
import os.path
import random
import warnings
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# PySpark: Configuración, DataFrame y funciones
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, DoubleType

# PySpark: ML y Pipeline
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    StringIndexer,
    VectorAssembler,
    Imputer,
    PCA,
    StandardScaler
)
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans as SKLearnKMeans

# Funciones de utilidad
from functools import reduce

# Ignorar advertencias deprecated
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
df = spark.sql("SELECT * FROM <datos>")
display(df)

#### Silhouette

In [ ]:
# Todas las columnas de tu DataFrame son numéricas, por lo que puedes usarlas directamente
feature_columns = df.columns

# Crear el VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Transformar el DataFrame para obtener la columna de características
df_features = assembler.transform(df)

In [ ]:
# Crear una lista vacía para almacenar las inercias
cs = []
silhouette_scores = []

# Probar diferentes valores de k (número de clusters), empezando en 2
for i in range(2, 20):
    kmeans = KMeans().setK(i).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(df_features)
    
    # Calcular la inercia y añadirla a la lista
    cs.append(model.summary.trainingCost)
    
    # Hacer predicciones
    predictions = model.transform(df_features)
    
    # Evaluar el modelo utilizando el coeficiente de silhouette
    evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette", distanceMeasure="squaredEuclidean")
    silhouette_score = evaluator.evaluate(predictions)
    silhouette_scores.append(silhouette_score)

# Trazar la curva de la inercia en función del número de clusters
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.plot(range(2, 20), cs, marker='o', linestyle='-', color='blue')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Criterio del Codo')

# Trazar la curva del coeficiente de silhouette en función del número de clusters
plt.subplot(1, 2, 2)
plt.plot(range(2, 20), silhouette_scores, marker='o', linestyle='-', color='red')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Coeficiente de Silhouette')
plt.title('Coeficiente de Silhouette para diferentes valores de k')

plt.tight_layout()
plt.show()